In [0]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
!git clone https://github.com/xelav/Insider-Threat-CERT /root/repo

import sys
sys.path.append("/root/repo")

fatal: destination path '/root/repo' already exists and is not an empty directory.


In [0]:
!pip install pytorch-ignite

In [0]:
%cd "/root/repo"
!git pull
%cd "{root_dir}"

In [0]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm.notebook import tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import SkipGram
from src.params import get_params
from src.dataset import SkipGramDataset, create_data_loaders
from src.w2v_trainer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO:

* Больше детерминизма



In [0]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(f'{root_dir}/Datasets/CERT_output')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

run_name = 'w2v/test2'

log_dir = output_dir / 'logs' / run_name
checkpoint_dir = output_dir / 'checkpoints' / run_name

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

if log_dir.is_dir():
    shutil.rmtree(log_dir)
if checkpoint_dir.is_dir():
    shutil.rmtree(checkpoint_dir)

# Train

In [0]:
actions = SkipGramDataset.prepare_dataset(output_dir / 'aggregated.pkl', min_length=50)
dataset = SkipGramDataset(actions, window_size=10)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1024)
device = 'cuda'

In [0]:
model = SkipGram(65, 30)
optimizer = optim.Adam(model.parameters())

train_engine = create_supervised_trainer_skipgram(
                                        model, optimizer, device=device,
                                        log_dir=log_dir.as_posix(),
                                        checkpoint_dir=checkpoint_dir,
                                       )

train_engine.run(dataloader, max_epochs=3)

In [0]:
%ls "{checkpoint_dir}"

final_model_7758.pth


In [0]:
from google.colab import files
files.download(checkpoint_dir / 'final_model_7758.pth') 

In [0]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

# Prediction exploration

https://projector.tensorflow.org/

In [0]:
import io

# encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for ind in range(65):
  vec = model.embed.weight[ind] # skip 0, it's padding.
  out_m.write(str(ind) + "\n")
  out_v.write('\t'.join([str(x) for x in vec.cpu().detach().numpy()]) + "\n")
out_v.close()
out_m.close()

In [0]:
from google.colab import files

files.download('vecs.tsv')
files.download('meta.tsv')